# Manipulating the Java Classpath and Imports

The magic `%classpath` adds jars to the kernel.  The magics `%import` and `%unimport` control which classes are visible by default in your code (in Groovy the regular `import` command is local to the cell where it runs.

These magics work in all the BeakerX JVM kernels.

## Example

This first cell shows that you get an error if you try to import a class not built-in to BeakerX:

In [ ]:
import com.example.Demo

Then load a jar  into the kernel:

In [ ]:
%classpath add jar ../resources/jar/demo.jar

After that, it imports and runs:

In [ ]:
import com.example.Demo
Demo demo = new Demo();
println demo.getObjectTest()

You can add multiple jars.  Wildcards also work, like they normally do in the classpath (in this case the code could say `../resources/jar/*`).

In [ ]:
%classpath add jar ../resources/jar/demo.jar
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

println com.example.Demo.staticTest();

import com.example.Demo

Demo demo = new Demo();
println demo.getObjectTest()

import com.beaker.BeakerXClasspathTest
BeakerXClasspathTest t = new BeakerXClasspathTest();
println com.beaker.BeakerXClasspathTest.staticTest;
println t.getObjectTest();
OutputCell.HIDDEN

The source code for these jars is in the repository. See [Demo.java](https://github.com/twosigma/beakerx/blob/master/kernel/demoProjects/demo/src/main/java/com/example/Demo.java), and [BeakerXClasspathTest.java](https://github.com/twosigma/beakerx/blob/master/kernel/demoProjects/BeakerXClasspathTest/src/main/java/com/beaker/BeakerXClasspathTest.java)

With no arguments the classpath magic prints-out all loaded jars:

In [ ]:
%classpath

There is also an API for getting the jars on the classpath:

In [ ]:
ClasspathManager.getJars()

## Loading a Single Dependency with Grapes and Maven

Groovy has a dependency manager called [Grape](http://docs.groovy-lang.org/latest/html/documentation/grape.html) built-in and you can access it as follows:

In [ ]:
@Grab(group='com.google.code.gson', module='gson', version='2.2.4')
import com.google.gson.GsonBuilder
new GsonBuilder().create().toJson([grab:[]], System.out)

The `%classpath` magic also supports loading from [maven central](https://search.maven.org):

In [ ]:
%classpath add mvn com.google.code.gson gson 2.2.4
new com.google.gson.GsonBuilder().create().toJson([magic:[]], System.out)

You can also use gradle-like syntax to load dependencies:

In [ ]:
%classpath add mvn com.sun.jersey:jersey-core:1.19.4
import com.sun.jersey.api.uri.UriBuilderImpl
return new UriBuilderImpl()
            .path("http://beakerx.com/")
            .path("documentation")
            .build();

## POM-Type Dependencies

Most dependencies are JARs, but a few such as [Jena are POMs](https://jena.apache.org/download/maven.html).  They can be loaded like this:

In [ ]:
%classpath add mvn org.apache.jena apache-jena-libs 3.6.0 pom
model = org.apache.jena.rdf.model.ModelFactory.createDefaultModel()

## POM-Classifier Dependencies

In [ ]:
%classpath add mvn org.platanios tensorflow_2.11 0.1.1 jar linux-cpu-x86_64
import org.platanios.tensorflow.api.core.Shape
Shape.matrix(2,5)

## Custom Repository

The `%classpath config resolver` gives you able to define custom repository:

In [ ]:
%classpath config resolver repository.spring.snapshot http://repo.spring.io/snapshot

In [ ]:
%classpath add mvn org.springframework spring-context 5.0.3.BUILD-SNAPSHOT

## Loading Multiple Dependencies with Maven

Use the `%%classpath` cell magic to load multiple dependencies at once. This magic creates a POM file containing all the dependencies listed in the cell, solves them together with Maven, and adds the result to the classpath.

In [ ]:
%%classpath add mvn
org.slf4j slf4j-api 1.7.25
org.slf4j slf4j-nop 1.7.25

## Loading Jars from a Dynamic Location

The above magics work on literal strings.  If you need to compute the location of a jar, use the dynamic classpath magic.  It has two versions one that works with a single string, and another that takes a list of strings:

In [ ]:
location = "../resources/jar/"

In [ ]:
%classpath add dynamic location + "demo.jar"

In [ ]:
%classpath add dynamic [location + "demo.jar", location + "BeakerXClasspathTest.jar"]

## Location of the Maven Cache and Using a Local Repository

Maven normally downloads jars to `~/.m2/repository/`.  But unfortunately this cache is [not safe for concurrent access](https://issues.apache.org/jira/browse/MNG-2802).
To avoid corrupting it, BeakerX by default uses its own Maven cache in the conda environment at `$(CONDA_PREFIX)/share/beakerx/maven`.  The `M2_HOME` environment variable overrides the default if set.

If you want to use a normal IDE to develop a library, publish it to your local Maven repository, and then load that library into BeakerX, you can, like this:

In [ ]:
%classpath config resolver mvnLocal

That is shorthand for a `%classpath config resolver local` magic with a `file:` path to the local repository.

## Clearing the Maven Cache

If you have painted yourself into a corner, you can clear the contents of the maven cache as follows.
This is useful for example if you have locally published [non-snapshot artifacts](https://maven.apache.org/guides/getting-started/index.html#What_is_a_SNAPSHOT_version). After resetting the cache, you will be prompted to restart your kernel.

In [ ]:
%classpath reset

## Import and Unimport

Normally `import` in Groovy only works in the cell where you use it.  To make a class import automatically into all cells, use `%import` magic.

In [ ]:
%import com.twosigma.beakerx.widget.IntSlider
w = new IntSlider()
w.value = 60
w

In [ ]:
w2 = new IntSlider()

In [ ]:
%unimport com.twosigma.beakerx.widget.IntSlider

In [ ]:
w3 = new IntSlider()

In [ ]:
%import static java.lang.Math.PI

In [ ]:
PI